# Divergent Strategy Exploration with defined Strategies 

In [13]:
# imports chapters 1 - 16
from chapter1 import calculate_stats
from chapter3 import standardDeviation
from chapter4 import (
    get_data_dict,
    create_fx_series_given_adjusted_prices_dict,
    calculate_variable_standard_deviation_for_risk_targeting_from_dict,
    calculate_position_series_given_variable_risk_for_dict,
)

from chapter5 import calculate_perc_returns_for_dict_with_costs
from chapter7 import calculate_forecast_for_ewmac
from chapter8 import apply_buffering_to_position_dict
from chapter9 import *

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## Strategy 9: Multiple Trendlines
Using the previously discussed methods of calculating exponential moving averages, we can create a strategy that uses multiple trendlines to determine when to buy and sell. We then use a capping method to the raw forecast of the trend signal combined with capital and risk management to determine the number of shares to buy or sell.


### Instrument Selection
- SP500
- US 10 Year Treasury Yield
- GAS FUTURES

### Spans of Trends
Typical spans are 16, 32, 64 as the fast. The slow is caculated by adding 2 logarthmic steps to the fast.
$$ slow = fast * 2^2 $$
$$ fast = 2^k, slow = 2^{k+2} $$

### Defining Strategy Relevant Functions
In the interest of saving time and space, I will import all other functions from previous chapters, however will include all functions that are relevant to the strategy below.

In [17]:
def calculate_position_dict_with_multiple_trend_forecast_applied(
    adjusted_prices_dict: dict,
    average_position_contracts_dict: dict,
    std_dev_dict: dict,
    fast_spans: list,
) -> dict:

    list_of_instruments = list(adjusted_prices_dict.keys())
    position_dict_with_trend_filter = dict(
        [
            (
                instrument_code,
                calculate_position_with_multiple_trend_forecast_applied(
                    adjusted_prices_dict[instrument_code],
                    average_position_contracts_dict[instrument_code],
                    stdev_ann_perc=std_dev_dict[instrument_code],
                    fast_spans=fast_spans,
                ),
            )
            for instrument_code in list_of_instruments
        ]
    )

    return position_dict_with_trend_filter


def calculate_position_with_multiple_trend_forecast_applied(
    adjusted_price: pd.Series,
    average_position: pd.Series,
    stdev_ann_perc: standardDeviation,
    fast_spans: list,
) -> pd.Series:

    forecast = calculate_combined_ewmac_forecast(
        adjusted_price=adjusted_price,
        stdev_ann_perc=stdev_ann_perc,
        fast_spans=fast_spans,
    )

    return forecast * average_position / 10


def calculate_combined_ewmac_forecast(
    adjusted_price: pd.Series,
    stdev_ann_perc: standardDeviation,
    fast_spans: list,
) -> pd.Series:

    all_forecasts_as_list = [
        calculate_forecast_for_ewmac(
            adjusted_price=adjusted_price,
            stdev_ann_perc=stdev_ann_perc,
            fast_span=fast_span,
        )
        for fast_span in fast_spans
    ]

    ### NOTE: This assumes we are equally weighted across spans
    ### eg all forecast weights the same, equally weighted
    all_forecasts_as_df = pd.concat(all_forecasts_as_list, axis=1)
    average_forecast = all_forecasts_as_df.mean(axis=1)

    ## apply an FDM
    rule_count = len(fast_spans)
    FDM_DICT = {1: 1.0, 2: 1.03, 3: 1.08, 4: 1.13, 5: 1.19, 6: 1.26}
    fdm = FDM_DICT[rule_count]

    scaled_forecast = average_forecast * fdm
    capped_forecast = scaled_forecast.clip(-20, 20)

    return capped_forecast

In [4]:
fast_spans = [16,32,64]

In [8]:
## Bringing in SP500 futures

adjusted_prices_dict, current_prices_dict = get_data_dict()
multipliers = dict(sp500=5, us10=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict)
capital = 1000000
idm = 1.5

instrument_weights = dict(sp500=0.5, us10=0.5)
cost_per_contract_dict = dict(sp500=0.875, us10=5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
    adjusted_prices=adjusted_prices_dict, current_prices=current_prices_dict
)

average_position_contracts_dict = (
    calculate_position_series_given_variable_risk_for_dict(
        capital=capital,
        risk_target_tau=risk_target_tau,
        idm=idm,
        weights=instrument_weights,
        std_dev_dict=std_dev_dict,
        fx_series_dict=fx_series_dict,
        multipliers=multipliers,
    )
)

The code above is primarily housekeeping work. It establishes risk targets, capital, and other hardcoded numbers as well as bringing in historical data.

In [27]:
position_contracts_dict = (
    calculate_position_dict_with_multiple_trend_forecast_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        std_dev_dict=std_dev_dict,
        fast_spans=fast_spans,
    )
)
    
buffered_position_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_dict = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)
## FIXED DEPRECATED WAY TO REACH POSITION DICT

/home/crott/Documents/AlgoGators/trend-following/chapter8.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_position = use_optimal_position[0]
/home/crott/Documents/AlgoGators/trend-following/chapter8.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  top_pos=upper_buffer[idx],
/home/crott/Documents/AlgoGators/trend-following/chapter8.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bot_pos=lower_buffer[idx],
/home/

In [28]:
print(calculate_stats(perc_return_dict["us10"]))

{'ann_mean': 0.07670379855733066, 'ann_std': 0.16948369149896184, 'sharpe_ratio': 0.4525733294981984, 'skew': -0.10320193199541221, 'avg_drawdown': 0.20774668802121116, 'max_drawdown': 0.5776817357807524, 'quant_ratio_lower': 2.501133102076955, 'quant_ratio_upper': 2.501133102076955}


In [29]:
position_contracts_df = pd.DataFrame.from_dict(position_contracts_dict) 
position_contracts_df.dropna(inplace=True)
position_contracts_df.head()

,sp500,us10
1982-09-16,-3.346614,0.321821
1982-09-17,-34.010185,0.464725
1982-09-20,-21.816418,0.714542
1982-09-21,18.079190,1.078945
1982-09-22,-0.612010,1.649941


In [30]:
perc_return_df = pd.DataFrame.from_dict(perc_return_dict)
perc_return_df.dropna(inplace=True)
perc_return_df.head(10)

,sp500,us10
1982-09-23,0.0,-0.000005
1982-09-24,-0.0,-0.000531
1982-09-27,0.0,0.000063
1982-09-28,-0.0,0.000558
1982-09-29,-0.0,-0.000125
1982-09-30,-0.0,0.000683
1982-10-01,0.0,0.002812
1982-10-04,0.0,-0.003938
1982-10-05,-0.0,0.000750
1982-10-06,0.0,0.002625


#### Plotting Histograms of Returns
- Plotting the histogram of returns for the SP500, US 10 Year Treasury Yield, and Gas Futures

In [35]:
# Plorring percentage returns of the SP500 and US10Y futures
plt.figure(1)
plt.hist(perc_return_df["sp500"],bins=500)
plt.title("SP500 Returns")
plt.xlabel("Date")
plt.ylabel("Percentage Return")
plt.figure(2)
plt.hist(perc_return_df["us10"], bins=500)
plt.title("US10 Returns")
plt.xlabel("Date")
plt.ylabel("Percentage Return")

plt.show()